# Applied Data Science Capstone

## Segmenting and Clustering Neighborhoods in Toronto

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import time
!conda install -c conda-forge geocoder --yes
import geocoder
!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup
import ssl
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
ssl._create_default_https_context = ssl._create_unverified_context
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    u

### Get the Contents of a Wikipedia Page

In [37]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
r.status_code

200

### Scrape the Contents

In [3]:
# instantiate a soup object with the response text
soup = BeautifulSoup(r.text, "html.parser")

# parse the table part of the response
postal_table = soup.find(class_="wikitable sortable")

### Transform the Data into a pandas Dataframe

In [38]:
# build the initial dataframe
table_rows = postal_table.find_all('tr')
row_values = []
for tr in table_rows:
    td = tr.find_all('td')
    row_text = [tr.text.strip() for tr in td if tr.text.strip()]
    if row_text:
        row_values.append(row_text)

toronto_df = pd.DataFrame(row_values, columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


### Clean the Dataframe

In [40]:
# ignore cells with a Borough that is 'Not assigned'
borough_df = toronto_df[toronto_df.Borough != 'Not assigned']

# combine neighborhoods with the same PostalCode into single row 
combined_df = borough_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index(drop=False)

combined_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [41]:
print(combined_df.shape)

(103, 3)


### END Part 1

### Get the Latitude & Longitude Coordinates of Each Neighborhood and Create a Dataframe

In [68]:
# use the link to the csv file to get latitude & longitude as geocoder appears unstable
url = 'http://cocl.us/Geospatial_data'
lat_long_df = pd.read_csv(url)

# create a new dataframe with the required columns
detailed_df = pd.DataFrame({'PostalCode':combined_df['PostalCode'], 
                            'Borough':combined_df['Borough'], 
                            'Neighborhood':combined_df['Neighborhood'], 
                            'Latitude':lat_long_df['Latitude'], 
                            'Longitude':lat_long_df['Longitude']})

print('Toronto has a total of {} boroughs and {} neighborhoods.'.format(len(detailed_df.Borough.unique()), len(detailed_df.Neighborhood.unique())))
print('\n')
detailed_df.head()

Toronto has a total of 10 boroughs and 98 neighborhoods.




,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### END Part 2

### Conduct Cluster Analysis

In [43]:
# analyze the number of postal codes in 'Toronto' boroughs
toronto_borough = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']

for tor in toronto_borough:
    print("{} has a total of {} postal codes.".format(tor, detailed_df[detailed_df['Borough'] == tor].PostalCode.count()))

Downtown Toronto has a total of 19 postal codes.
Central Toronto has a total of 9 postal codes.
West Toronto has a total of 6 postal codes.
East Toronto has a total of 5 postal codes.


In [45]:
# create a new dataframe for the cluster analysis of 'Toronto' boroughs
d_t = detailed_df[detailed_df['Borough'] == 'Downtown Toronto']
c_t = detailed_df[detailed_df['Borough'] == 'Central Toronto']
w_t = detailed_df[detailed_df['Borough'] == 'West Toronto']
e_t = detailed_df[detailed_df['Borough'] == 'East Toronto']

combined = pd.concat([d_t, c_t, w_t, e_t], sort=False)
toronto_dataframe = combined.reset_index(drop=True)

print(toronto_dataframe.shape)
toronto_dataframe.head(10)

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568
9,M5J,Downtown Toronto,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752


In [69]:
# get the latitude & longitude values of Toronto
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Coordinates of Toronto are {}, {}. '.format(latitude, longitude))

Coordinates of Toronto are 43.6534817, -79.3839347. 


### Create a Map of Toronto with its Neighborhoods Superimposed on Top

In [70]:
# create a map of toronto using latitude & longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers
for lat, lng, borough, neighborhood in zip(toronto_dataframe['Latitude'], toronto_dataframe['Longitude'], toronto_dataframe['Borough'], toronto_dataframe['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### Explore the Neighborhoods Utilizing the Foursquare API and Segment Them

In [48]:
# access Foursquare APIs
client_id = 'I0WSDFW5UEWAMMEATZRZL5B1VMKJV5TDZIAVJSGS1EVVBFST'
client_secret = 'KP1KVLWC4NMAVJSCPBODF0RIFXGPKZ2KDCIRMOO2TNMEO03X'
version = '20200504'

In [49]:
# first neighborhood in toronto_dataframe
toronto_dataframe.loc[0, 'Neighborhood']

'Rosedale'

In [71]:
# get Rosedale's latitude & longitude values
rosedale_latitude = toronto_dataframe.loc[0, 'Latitude']
rosedale_longitude = toronto_dataframe.loc[0, 'Longitude']

print('Rosedale\'s latitude and longitude values are {}, {}.'.format(rosedale_latitude, rosedale_longitude))

Rosedale's latitude and longitude values are 43.6795626, -79.37752940000001.


In [51]:
# get the top 100 venues that are in Rosedale within a radius of 500 meteres
no_of_venues = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id, client_secret, version, rosedale_latitude, rosedale_longitude, radius, no_of_venues)

response = requests.get(url).json()
response

{'meta': {'code': 200, 'requestId': '5eafdcd45fb726001b857ebe'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aff2d47f964a520743522e3',
       'name': 'Rosedale Park',
       'location': {'address': '38 Scholfield Ave.',
        'crossStreet': 'at Edgar Ave.',
        'lat': 43.68232820227814,
        'lng': -79.37893434347683,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68232820227814,
          'lng': -79.37893434347683}],
        'distance': 32

In [52]:
# function to extract the venue's category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
# clean the response json and structure it into pandas dataframe
venues = response['response']['groups'][0]['items']
# flatten json
nearby_venues = json_normalize(venues)
# filter only needed columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


### Explore All Neighborhoods and its Venues

In [54]:
# explore all the neighborhoods of four different boroughs using Foursquare APIs
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id, client_secret, version, lat, lng, radius, no_of_venues)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name, lat, lng, 
            v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
                            'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [55]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues
toronto_venues = getNearbyVenues(names=toronto_dataframe['Neighborhood'],
                                 latitudes=toronto_dataframe['Latitude'],
                                 longitudes=toronto_dataframe['Longitude'])

Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District / Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Queen's Park / Ontario Provincial Government
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / The Junction South
Parkdale / Roncesvalles
R

In [64]:
# check how many venues were returned overall
print(toronto_venues.shape)
print('\n')
toronto_venues.head(10)

(1623, 7)




,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,St. James Town / Cabbagetown,43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
5,St. James Town / Cabbagetown,43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
6,St. James Town / Cabbagetown,43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
7,St. James Town / Cabbagetown,43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
8,St. James Town / Cabbagetown,43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
9,St. James Town / Cabbagetown,43.667967,-79.367675,Merryberry Cafe + Bistro,43.666630,-79.368792,Café


In [57]:
# check the number of venues for each neighborhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Berczy Park                                                                                                          57
Brockton / Parkdale Village / Exhibition Place                                                                       23
Business reply mail Processing Centre                                                                                18
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport     17
Central Bay Street                                                                                                   63
Christie                                                                                                             17
Church and Wellesley                                                                                                 78
Commerce Court / Victoria Hotel                                                                                     100
Davisville                 

In [58]:
# check the number of unique categories of venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


### Do One-hot Encoding All Neighborhoods based on the Venue Category

In [60]:
# use one-hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add the neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move the neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(1623, 231)

### Group Rows by Neighborhood and Take the Mean of the Frequency of Occurrence of Each Category

In [61]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head(5)

(39, 231)


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Business reply mail Processing Centre,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.0,0.0,0.0


### Each Neighborhood with Top 5 Common Venues

In [62]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("[--------"+hood+"--------]")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

[--------Berczy Park--------]
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3          Restaurant  0.04
4  Seafood Restaurant  0.04


[--------Brockton / Parkdale Village / Exhibition Place--------]
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2       Nightclub  0.09
3     Coffee Shop  0.09
4   Grocery Store  0.04


[--------Business reply mail Processing Centre--------]
              venue  freq
0       Yoga Studio  0.06
1     Auto Workshop  0.06
2       Pizza Place  0.06
3        Comic Shop  0.06
4  Recording Studio  0.06


[--------CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport--------]
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4  Sculpture Garden  0.06


[--------Central Bay Street--------]
                venue  freq
0         Coffee Shop

### Create a Dataframe that Has Top 10 Venues for Each Neighborhood

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
# create a dataframe to display top 10 venues for each neighborhood
num_top_venues = 10
indicators = ['st', 'nd', 'rd', 'th']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
         columns.append('{}th Most Common Venue'.format(ind+1))
        
# create the new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.shape

(39, 11)

### Run K-Means to Cluster the Neighborhoods

In [66]:
# set the number of clusters
kclusters = 4
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_init=300 , n_clusters=kclusters, random_state=5).fit(toronto_grouped_clustering)

#check cluster lables generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0],
      dtype=int32)

### Create a New Dataframe that Includes the Clusters

In [30]:
# add cluster labels
neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_dataframe

# merge toronto_grouped with toronto_dataframe to add latitude & longitue for each neighborhood
toronto_merged = toronto_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675,0,Coffee Shop,Pub,Café,Bakery,Restaurant,Park,Italian Restaurant,Pizza Place,Playground,Pharmacy
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Gay Bar,Mediterranean Restaurant,Pub,Hotel,Smoke Shop
3,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Restaurant,Chocolate Shop
4,M5B,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Restaurant,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant


### Visualize the Clusters

In [67]:
# create a map
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color schemes
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=9,
    popup=label,
    colors=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=2.0).add_to(cluster_map)

cluster_map

### Examine Each Cluster and Determine the Discriminating Venue Categories that Distinguish Each Cluster

In [33]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,-79.367675,0,Coffee Shop,Pub,Café,Bakery,Restaurant,Park,Italian Restaurant,Pizza Place,Playground,Pharmacy
2,Downtown Toronto,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Gay Bar,Mediterranean Restaurant,Pub,Hotel,Smoke Shop
3,Downtown Toronto,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Restaurant,Chocolate Shop
4,Downtown Toronto,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Restaurant,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant
5,Downtown Toronto,-79.375418,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Hotel,Restaurant,Gym,Italian Restaurant,Department Store
6,Downtown Toronto,-79.373306,0,Coffee Shop,Cocktail Bar,Beer Bar,Cheese Shop,Restaurant,Café,Bakery,Seafood Restaurant,Bistro,Jazz Club
7,Downtown Toronto,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Middle Eastern Restaurant,Bar,Thai Restaurant,Burger Joint,Fried Chicken Joint
8,Downtown Toronto,-79.384568,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,Gym,Clothing Store,Hotel,Bakery,Steakhouse
9,Downtown Toronto,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Brewery,Sporting Goods Shop,Italian Restaurant,Scenic Lookout,Fried Chicken Joint
10,Downtown Toronto,-79.381576,0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Salad Place,Deli / Bodega,American Restaurant,Tea Room


In [34]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Central Toronto,-79.416936,1,Garden,Women's Store,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [35]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,-79.38316,2,Park,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [36]:
# Cluster - 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.377529,3,Park,Playground,Trail,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
19,Central Toronto,-79.388790,3,Swim School,Park,Construction & Landscaping,Bus Line,Women's Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
26,Central Toronto,-79.411307,3,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
